# Preprocessing

### Not essential for the hacknight!

Contains functions used for processing float geotiffs to small array files.

Original geotiffs aren't included in repo

In [9]:
import gdal
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as img

In [15]:
def readTiff(fp):
    im = gdal.Open(fp)
    im = im.ReadAsArray()
    im = np.swapaxes(im,0,2)
    return np.swapaxes(im,0,1)

def image_histogram_equalization(image):
    """ Image histogram equlisation"""
    # from http://www.janeriksolem.net/2009/06/histogram-equalization-with-python-and.html

    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), 255, normed=False)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = (255-1) * cdf / cdf[-1] # normalize

    # use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)
    
    return image_equalized.reshape(image.shape).astype('uint8')
    
def makeRGB(im):
    im0 = image_histogram_equalization(im[:,:,0])
    im1 = image_histogram_equalization(im[:,:,1])
    r = image_histogram_equalization((im[:,:,1]/im[:,:,0]))
    rgb = np.dstack((im1,im0,r))
    return rgb

def convert_to_jpg(fnames,directory):
    for name in fnames:
        path = os.path.join(directory,name)
        nname = os.path.join(directory,name.split('.')[0]+'.jpg')
        im = img.fromarray(np.load(path))
        im.save(nname)
    return 'done'

In [16]:
root = '../data/'
fnames = ['SARim1_16_09_2017.npy',
        'SARim2_22_09_2017.npy',
        'SARim3_22_10_2017.npy']

In [17]:
convert_to_jpg(fnames,root)

'done'